In [5]:
from model import CongestionLearnableEmbedding, CongestionWrapperEncoder
import torch

cong = torch.randn(10, 1)

x = CongestionWrapperEncoder(CongestionLearnableEmbedding, torch.randn(10, 1), )


ModuleNotFoundError: No module named 'parameters'

In [6]:
import torch
import torch.nn.functional as F

num_nodes = 10

sparsity = 0.3

adjacency_matrix = torch.rand(num_nodes, num_nodes)
adjacency_matrix = (adjacency_matrix < sparsity).float()


adjacency_matrix = adjacency_matrix * (1 - torch.eye(num_nodes))

print("Random Adjacency Matrix:")
print(adjacency_matrix)

Random Adjacency Matrix:
tensor([[0., 1., 0., 1., 0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1., 0., 1., 0., 1., 0.],
        [0., 1., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 1., 1., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 1., 0., 0., 0., 0.]])


In [10]:
import torch
from torch_geometric.nn import GATConv


batch_size = 3
num_nodes = 10
num_features = 16

x = torch.randn(batch_size, num_nodes, num_features)

# edge_index = torch.randint(0, num_nodes, (2, num_nodes * 3))
edge_index = torch.stack([adjacency_matrix for x in range(3)])
print(edge_index)

in_channels = num_features
out_channels = 8
num_heads = 2

gat_layer = GATConv(in_channels, out_channels, heads=num_heads)

output_tensor = gat_layer(x, edge_index)

print("Shape of output_tensor:", output_tensor.shape)

tensor([[[0., 1., 0., 1., 0., 0., 0., 0., 1., 1.],
         [0., 0., 0., 0., 1., 0., 1., 0., 1., 0.],
         [0., 1., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 1., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 1., 1., 0., 1., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [1., 0., 1., 0., 0., 1., 0., 0., 0., 0.]],

        [[0., 1., 0., 1., 0., 0., 0., 0., 1., 1.],
         [0., 0., 0., 0., 1., 0., 1., 0., 1., 0.],
         [0., 1., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 1., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 1., 1., 0., 1., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [1., 0., 1., 0., 0.,

AssertionError: Static graphs not supported in 'GATConv'

In [11]:
import torch
import torch.nn as nn
from torch_geometric.nn import GATConv
from torch_geometric.data import Data


edge_index = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]], dtype=torch.long)
x = torch.rand(3, 16)  

data = Data(x=x, edge_index=edge_index)


class GraphAttentionLayer(nn.Module):
    def __init__(self, in_channels, out_channels, heads=1, dropout=0.6):
        super().__init__()
        self.gat_conv = GATConv(in_channels, out_channels, heads=heads, dropout=dropout)

    def forward(self, data):
        x = self.gat_conv(data.x, data.edge_index)
        return x


input_features = 16
output_features = 8
num_heads = 2

graph_attention_layer = GraphAttentionLayer(input_features, output_features, heads=num_heads)

output = graph_attention_layer(data)

print(output)

tensor([[ 1.1303, -0.0318,  1.9701,  0.0700,  0.3179,  0.9592, -0.6862,  0.0443,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.2974,  0.1202,  0.3389, -0.5510,  0.0355,  0.0388,  0.5925,  0.1925],
        [ 1.8897, -0.7915,  3.3366,  0.5900, -0.2979,  1.6662, -1.6858,  0.4432,
          0.7429,  0.0752,  0.2040, -0.7052,  0.4324, -0.2763,  1.1395,  0.6563]],
       grad_fn=<AddBackward0>)


In [12]:
import torch
import torch.nn as nn
from torch_geometric.nn import GATConv
from torch_geometric.data import Data

# Example: Batch of graphs with 3 graphs, each having 4 nodes and 16 features per node
batch_size = 3
num_nodes = 4
num_features = 16

edge_index = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]], dtype=torch.long)
x = torch.rand(batch_size, num_nodes, num_features)

data = Data(x=x, edge_index=edge_index)

class GraphAttentionLayer(nn.Module):
    def __init__(self, in_channels, out_channels, heads=1, dropout=0.6):
        super().__init__()
        self.gat_conv = GATConv(in_channels, out_channels, heads=heads, dropout=dropout)

    def forward(self, data):
        x = self.gat_conv(data.x, data.edge_index)
        return x

input_features = 16
output_features = 8
num_heads = 2

graph_attention_layer = GraphAttentionLayer(input_features, output_features, heads=num_heads)

# Forward pass through the GAT layer for the entire batch
output = graph_attention_layer(data)

print(output)


AssertionError: Static graphs not supported in 'GATConv'